using DFID

In [2]:
from typing import List, Tuple, Optional, Dict, Set

In [3]:
State = Tuple[int, ...]
GOAL: State = (1,2,3,4,5,6,7,8,0)

def is_solvable(s: State) -> bool:
 # inversion count method
 arr = [x for x in s if x]
 inv = sum(arr[i] > arr[j] for i in range(len(arr)) for j in range(i+1, len(arr)))
 return inv % 2 == 0



In [4]:
def neighbors(s: State) -> List[Tuple[State,str]]:
 i = s.index(0); r, c = divmod(i, 3)
 def swap(a, b):
     lst = list(s); lst[a], lst[b] = lst[b], lst[a]; return tuple(lst)
 moves = []
 if r > 0: moves.append((swap(i, i-3), 'U'))
 if r < 2: moves.append((swap(i, i+3), 'D'))
 if c > 0: moves.append((swap(i, i-1), 'L'))
 if c < 2: moves.append((swap(i, i+1), 'R'))
 return moves

In [5]:
def dls(start: State, limit: int, stats: Dict[str,int]) -> Tuple[Optional[List[str]], bool]:
 cutoff = False
 stack = [(start, 0)]
 parent = {start: None}
 move_to = {start: None}
 pathset = {start}

 while stack:
     s, d = stack.pop()
     if s == GOAL:
         path = []
         while move_to[s]:
             path.append(move_to[s])
             s = parent[s]
         return path[::-1], False
     if d == limit:
         cutoff = True
         continue
     for nxt, m in neighbors(s):
         stats["gen"] += 1
         if nxt in pathset:
             stats["cycle"] += 1
             continue
         parent[nxt], move_to[nxt] = s, m
         pathset.add(nxt)
         stack.append((nxt, d+1))
     stats["exp"] += 1
     pathset.remove(s)
 return None, cutoff

In [6]:
def dfid(start: State, maxd=40):
 if not is_solvable(start):
     return None, {"solvable": False}
 stats = {"exp":0, "gen":0, "cycle":0}
 for limit in range(maxd+1):
     st = {"exp":0, "gen":0, "cycle":0}
     path, cut = dls(start, limit, st)
     for k in stats: stats[k] += st[k]
     if path:
         return path, {**stats, "solvable": True, "depth": limit, "sol_depth": len(path)}
     if not cut:
         return None, {**stats, "solvable": False, "depth": limit}
 return None, {**stats, "solvable": False}

In [7]:

def pretty(s: State) -> str:
 return "\n".join(" ".join(f"{v:2}" if v else "  " for v in s[i:i+3]) for i in range(0,9,3))


def apply_moves(s: State, moves: List[str]) -> State:
 for m in moves:
     s = [n for n,m2 in neighbors(s) if m2 == m][0]
 return s

In [9]:
start = (1,2,3,
      4,0,5,
      6,7,8)

print("Start:\n", pretty(start), "\n")

path, stats = dfid(start, 20)

if path: 
 print(f"Solution depth {len(path)}")
 print("Moves:", ' '.join(path))
 s = start
 for i, m in enumerate(path, 1):
     s = apply_moves(s, [m])
     print(f"\nStep {i}: {m}\n{pretty(s)}")
 print("\nStats:", stats)
else:
 print("No solution found within depth limit.", stats)

Start:
  1  2  3
 4     5
 6  7  8 



MemoryError: 